In [4]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPool2D, Flatten
import tensorflow.keras as keras
from skimage import feature
import re

In [5]:
files = [file for file in glob.glob('Projeto/Fotos/H/*/*.jpg')]

files = [file.replace('\\', '/') for file in files]


In [6]:
labels = []

for i, file in enumerate(files):
    
    img = cv2.imread(file) ### ler a imagem
        
    img_512 = cv2.resize(img,(512, 512),interpolation=cv2.INTER_AREA)
        
    img = cv2.cvtColor(img_512, cv2.COLOR_BGR2RGB)
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
       
    label = file.split(sep='/')[3]
    print(label)    
    
    name = file.split(sep='/')[-1]
    print(name)    
    
    #cv2.imwrite(f'Projeto/' + label + '/' + name + '.png', img_gray)
    labels.append(label)

In [7]:
files = [file for file in glob.glob('Projeto/Fotos/size/*.png')]
files = [file.replace('\\', '/') for file in files]
files.sort()

dataset = []
for file in files:
    img = cv2.imread(file)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    dataset.append(img_gray)

In [8]:
len(labels)

0

In [9]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.33, stratify=labels, random_state=42)

X_train = np.asarray(X_train)

X_test = np.asarray(X_test)

print('tamanho do dataset de treino:', X_train.shape)

print('tamanho do dataset de teste:', X_test.shape)

ValueError: With n_samples=0, test_size=0.33 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
plt.imshow(X_train[4]);
print(y_train[4])

In [ ]:
X_train_rf = X_train.reshape(X_train.shape[0], -1)

X_test_rf = X_test.reshape(X_test.shape[0], -1)

print(X_train_rf.shape)
print(X_test_rf.shape)

In [ ]:
clf = RandomForestClassifier()

clf.fit(X_train_rf, y_train) 

y_pred_rf = clf.predict(X_test_rf)

In [ ]:
#print(classification_report(X_train_rf, y_train))
print(classification_report(y_test, y_pred_rf))

In [ ]:
confusion_matrix(y_test, y_pred_rf)

In [ ]:
plt.figure(figsize=[10, 10])

fig = ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rf)

plt.show(fig)

In [ ]:
img_512 = cv2.resize(teste,(512, 512),interpolation=cv2.INTER_AREA)

plt.figure(figsize=[10, 10])

plt.imshow(img_256);

In [ ]:
print(labels)

In [ ]:
plt.imshow(labels[0])